**SA433A &#x25aa; Data Wrangling and Visualization &#x25aa; Fall 2024**

# Lesson 12. Introduction to Data Wrangling &mdash; The Pandas DataFrame and Series

## Overview


- **Data wrangling** (sometimes called **data munging**) is the act of transforming or manipulating data into the right form you need for analysis

- This might involve:
    - filtering or rearranging data
    - creating new data based on the existing data
    - merging different data sets together

- For example, the datasets you've been given for Project 3 actually came from 4 different datasets, managed by different parties

- Data wrangling isn't glamorous, but it is very necessary!
    - Real data sets, unlike the data sets you often encounter in class, are messy
    - It's pretty common to spend more time on wrangling data than data analysis!

- Starting with this lesson, we'll learn how to wrangle data sets with [**Pandas**](https://pandas.pydata.org/), a Python library for data manipulation and analysis


- Pandas is powerful and vast, and there are typically many ways to do a particular task with Pandas


- We will focus on a *curated* approach to using a *subset* of the Pandas library that will be sufficient to perform most data wrangling tasks

## In this lesson...

- The DataFrame object and the index


- The Series object


- Arithmetic operations, broadcasting, and alignment


- Adding a column to a DataFrame

<hr style="border-top: 2px solid gray; margin-top: 1px; margin-bottom: 1px"></hr>

## The DataFrame object and the index

- Let's import Pandas &mdash; we don't need Altair for this lesson

In [1]:
import pandas as pd

- We've already seen the **DataFrame** in this class, as a way of representing tabular data with rows and columns


- In particular, we know how to read a CSV file with tabular data into a DataFrame


- For example, we can read in the contents of `data/small_grades.csv` with `pd.read_csv()`:

In [2]:
df = pd.read_csv('data/small_grades.csv')

- Recall that we can get the number of rows and columns of `df` with `df.shape`:

In [3]:
df.shape

(5, 8)

* Also recall that we can get some basic information about `df` with `df.info()`:

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   student_id  5 non-null      int64 
 1   last_name   5 non-null      object
 2   first_name  5 non-null      object
 3   quiz1       5 non-null      int64 
 4   quiz2       5 non-null      int64 
 5   quiz3       5 non-null      int64 
 6   exam1       5 non-null      int64 
 7   exam2       5 non-null      int64 
dtypes: int64(6), object(2)
memory usage: 452.0+ bytes


- As we can see from the output, we have 8 columns in this dataset


- Possible `dtype`s: 

| `dtype` | Usage |
| :- | :- |
| `object` | text (usually...) |
| `int64` | integer numbers |
| `float64` | floating point numbers |
| `bool` | Booleans: True/False values |
| `datetime64` | date and time values |
| `timedelta[ns]` | differences between two `datetime64`s |
| `category` | finite list of text values |


- We can preview the first 5 rows of `df` with `df.head()`:

In [5]:
df.head()

,student_id,last_name,first_name,quiz1,quiz2,quiz3,exam1,exam2
0,3208,Schmitt,Logan,89,81,85,97,75
1,2476,Burks,Keanan,88,76,63,50,96
2,1434,Nguyen,Janine,52,51,89,88,82
3,9812,Parry,Kristina,51,82,96,62,90
4,2369,Waters,Mikaela,90,62,53,80,61


- The left-most column is called the **index** of the DataFrame


- The index assigns a *label* to each row/observation


- The index does *not* count as a column/variable of the DataFrame

- Often, it will be useful to set one of the existing columns as the index of the DataFrame


- For example, we can set the index to be the values in the `student_id` column with the `.set_index()` DataFrame method, like this:

In [6]:
# Solution
df = df.set_index(keys='student_id', drop=True)

- Some notes about `.set_index()`:
    - `keys=...` specifies which column to use as the index
    - `drop=...` specifies whether the column used as the index should be dropped from the DataFrame
    - Note that `.set_index()` does *not* change the DataFrame in place, instead it returns a DataFrame
        - Therefore, we need to grab the DataFrame that `.set_index()` returns in order to use the newly created index
    - [Documentation for `.set_index()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.set_index.html)

- Looking at the new `df`, we see that the index now consists of the values from the `student_id` column:

In [7]:
df.head()

,last_name,first_name,quiz1,quiz2,quiz3,exam1,exam2
student_id,,,,,,,
3208,Schmitt,Logan,89,81,85,97,75
2476,Burks,Keanan,88,76,63,50,96
1434,Nguyen,Janine,52,51,89,88,82
9812,Parry,Kristina,51,82,96,62,90
2369,Waters,Mikaela,90,62,53,80,61


- We'll come back to how the index is used shortly

<hr style="border-top: 2px solid gray; margin-top: 1px; margin-bottom: 1px"></hr>

## The Series object

- Each column of a DataFrame is a **Series** object


- We can access a column/Series of a DataFrame using Python dictionary notation, with the column name as the key


- For example, we can get the `last_name` column from `df` like this:

In [8]:
# Solution
df['last_name']

student_id
3208    Schmitt
2476      Burks
1434     Nguyen
9812      Parry
2369     Waters
Name: last_name, dtype: object

- Note that the Series object also comes with an index, just like a DataFrame

<hr style="border-top: 2px solid gray; margin-top: 1px; margin-bottom: 1px"></hr>

## Arithmetic operations, broadcasting, and alignment

- We can perform arithmetic operations between columns/Series using the usual operators: `+`, `-`, `*`, `/`


- For example, we can compute the difference between each student's Exam 1 and Exam 2 scores like this:

In [9]:
# Solution
df['exam1'] - df['exam2']

student_id
3208    22
2476   -46
1434     6
9812   -28
2369    19
dtype: int64

- A few notes before moving on:
    - The result of `+`, `-`, `*`, `/` between two columns or Series objects is another Series object
    - Arithmetic operations are element-wise (i.e., *not* like matrix algebra)

- We can also perform arithmetic operations between a column/Series and a scalar


- For example, we can compute each student's exam average like this:

In [10]:
# Solution
(df['exam1'] + df['exam2']) / 2

student_id
3208    86.0
2476    73.0
1434    85.0
9812    76.0
2369    70.5
dtype: float64

- Note that each element of the Series `df['exam1'] + df['exam2']` is divided by 2


- This is called **broadcasting**: how operations are performed between data of different shapes


- Broadcasting rules can get quite complex


- Another important note: Pandas automatically **aligns** the data by the *index label* when performing arithmetic operations


- To illustrate this, let's read in another dataset containing Quiz 1 bonus points, found in `data/quiz1_bonus.csv`:

In [11]:
bonus_df = pd.read_csv('data/small_quiz1_bonus.csv')
bonus_df.head()

,student_id,quiz1_bonus
0,9812,6
1,1434,3
2,3208,4
3,2369,10
4,2476,0


- Like with `df`, let's set the index of `bonus_df` to `student_id`:

In [12]:
# Solution
bonus_df = bonus_df.set_index(keys='student_id')
bonus_df.head()

,quiz1_bonus
student_id,
9812,6
1434,3
3208,4
2369,10
2476,0


- Note that the student ID numbers are the same as those in `df`, but in a different order


- Let's see what happens when we add these bonus points to the original Quiz 1 scores:

In [13]:
# Solution
df['quiz1'] + bonus_df['quiz1_bonus']

student_id
1434     55
2369    100
2476     88
3208     93
9812     57
dtype: int64

- Pandas correctly adds the bonus points to the original Quiz 1 scores with *matching index labels*

<hr style="border-top: 2px solid gray; margin-top: 1px; margin-bottom: 1px"></hr>

## Adding a column to a DataFrame

- One way of adding a column named `new_col` to a DataFrame `df` is:

    ```python
    df['new_col'] = ...
    ```

- For example, let's do the following:
    1. Add a new column to `df` containing the Quiz 1 bonus points
    2. Add a new column to `df` containing the adjusted Quiz 1 score (original score + bonus points)

In [14]:
# Solution
df['quiz1_bonus'] = bonus_df['quiz1_bonus']
df['quiz1_adjusted'] = df['quiz1'] + df['quiz1_bonus']
df.head()

,last_name,first_name,quiz1,quiz2,quiz3,exam1,exam2,quiz1_bonus,quiz1_adjusted
student_id,,,,,,,,,
3208,Schmitt,Logan,89,81,85,97,75,4,93
2476,Burks,Keanan,88,76,63,50,96,0,88
1434,Nguyen,Janine,52,51,89,88,82,3,55
9812,Parry,Kristina,51,82,96,62,90,6,57
2369,Waters,Mikaela,90,62,53,80,61,10,100


- Note that the Quiz 1 bonus points are matched correctly with the corresponding `student_id`


- We will learn another, typically more convenient way to add columns to a DataFrame in a later lesson 

<hr style="border-top: 2px solid gray; margin-top: 1px; margin-bottom: 1px"></hr>

## Problems

### Problem 0

The following problems are based on the CSV file `data/grades.csv` located in the same folder as this notebook. Read the CSV file into a DataFrame below. Display the top 5 rows of the DataFrame.

In [15]:
# Solution
df = pd.read_csv('data/grades.csv')
df.head()

,student_id,lastname,firstname,quiz1,quiz2,quiz3,quiz4,exam1,exam2
0,2603,Conway,Romeo,19,29,20,30,87,50
1,6435,Livingston,Naseem,17,16,14,25,55,83
2,6754,Clark,Remy,15,30,11,24,60,95
3,3032,Carpenter,Zubair,16,13,18,14,91,71
4,2715,Guerra,Samantha,13,15,17,28,48,98


### Problem 1

How many students are in this dataset?

In [16]:
# Solution
df.shape[0]

49

### Problem 2

Compute each student's exam average as a percentage (a fraction between 0 and 1). The maximum score on each exam is 100 points. Each exam is equally weighted in a student's exam average. Add a column containing the exam averages to the DataFrame. Check your work by displaying the top 5 rows of the DataFrame.

In [17]:
# Solution
df['exam_avg'] = ((df['exam1'] / 100) + (df['exam2'] / 100)) / 2
df.head()

,student_id,lastname,firstname,quiz1,quiz2,quiz3,quiz4,exam1,exam2,exam_avg
0,2603,Conway,Romeo,19,29,20,30,87,50,0.685
1,6435,Livingston,Naseem,17,16,14,25,55,83,0.690
2,6754,Clark,Remy,15,30,11,24,60,95,0.775
3,3032,Carpenter,Zubair,16,13,18,14,91,71,0.810
4,2715,Guerra,Samantha,13,15,17,28,48,98,0.730


### Problem 3

Compute each student's quiz average as a percentage (fraction between 0 and 1). The maximum score on Quizzes 1 and 3 is 20 points; the maximum score on Quizzes 2 and 4 is 30 points. Each quiz is equally weighted in a student's quiz average. Add a column containing the quiz averages to the DataFrame. Check your work by displaying the top 5 rows of the DataFrame.

In [18]:
# Solution
df['quiz_avg'] = ((df['quiz1'] / 20) + (df['quiz2'] / 30) + (df['quiz3'] / 20) + (df['quiz4'] / 30)) / 4
df.head()

,student_id,lastname,firstname,quiz1,quiz2,quiz3,quiz4,exam1,exam2,exam_avg,quiz_avg
0,2603,Conway,Romeo,19,29,20,30,87,50,0.685,0.979167
1,6435,Livingston,Naseem,17,16,14,25,55,83,0.690,0.729167
2,6754,Clark,Remy,15,30,11,24,60,95,0.775,0.775000
3,3032,Carpenter,Zubair,16,13,18,14,91,71,0.810,0.650000
4,2715,Guerra,Samantha,13,15,17,28,48,98,0.730,0.733333


### Problem 4

Compute each student's course grade as a weighted average of each student's quiz average and exam average: quizzes are worth 35\%, exams are worth 65\%. Add a column containing the course grades to the DataFrame. 


Check your work by displaying the top 5 rows of the DataFrame. You should find that Romeo Conway has a course grade of about 78.8%, Naseem Livingston has a course grade of 70.4%, and Remy Clark has a course grade of 77.5%.

In [19]:
# Solution
df['course_grade'] = 0.35 * df['quiz_avg'] + 0.65 * df['exam_avg']
df.head()

,student_id,lastname,firstname,quiz1,quiz2,quiz3,quiz4,exam1,exam2,exam_avg,quiz_avg,course_grade
0,2603,Conway,Romeo,19,29,20,30,87,50,0.685,0.979167,0.787958
1,6435,Livingston,Naseem,17,16,14,25,55,83,0.690,0.729167,0.703708
2,6754,Clark,Remy,15,30,11,24,60,95,0.775,0.775000,0.775000
3,3032,Carpenter,Zubair,16,13,18,14,91,71,0.810,0.650000,0.754000
4,2715,Guerra,Samantha,13,15,17,28,48,98,0.730,0.733333,0.731167


<hr style="border-top: 2px solid gray; margin-top: 1px; margin-bottom: 1px"></hr>

## Notes and sources

- From the [Pandas User Guide](https://pandas.pydata.org/docs/user_guide/index.html):
    - [Intro to data structures](https://pandas.pydata.org/docs/user_guide/dsintro.html)
    - [Essential basic functionality](https://pandas.pydata.org/docs/user_guide/basics.html)